# **ETL**

#### Realizaremos el proceso de Extracción, Transformación y Carga (`ETL`) de los datos necesarios para el proyecto, obteniéndolos de varias fuentes confiables.

---

In [1]:
# Librerias a utilizar
import requests
import pandas as pd
import calendar
import io
import datetime
import json
import re
from bs4 import BeautifulSoup

---

### *Datos mundiales de terremotos de 1970 a 2023*

In [30]:
# Extraemos los datos mediante la API de la USGS

# Creamos lista donde almacenaremos todos los datos
all_data = []

# Iteramos para extraer los datos de los años de interes, en este caso de 1970 a 2023 filtrando los mayores a magnitud 4
for i in range(1970, 2023):
    year = i
    for mes in range (1, 13):
        # Tomamos el último dia de cada mes
        last_day = calendar.monthrange(year, mes)[1]
        starttime = f"{year}-{mes}-01"
        endtime = f"{year}-{mes}-{last_day}"
        # El url de la api con los parámetros, donde filtramos buscando magnitudes mayores a 4
        url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={starttime}&endtime={endtime}&minmagnitude=4"
        # Envía la solicitud HTTP a la API de USGS y procesa los datos de respuesta
        response = requests.get(url)
        data = json.loads(response.text)
        data = data["features"]
        # Itera sobre cada objeto "Feature" en la lista "data"
        for feature in data:
            # Accede a la parte "properties" de cada objeto que es donde se encuentran los datos de interés
            properties = feature['properties']
            properties['year'] = year
            properties['month'] = mes
            # Agregamos los datos a la lista
            all_data.append(properties)
   
    
# Realizamos la misma extracción, pero para el año 2023 de forma actualizada siempre al tiempo actual
# Tomamos el mes actual
mes_actual = datetime.datetime.now().month
# Iteramos por mes, con tope en el mes actual
for mes in range (1, mes_actual +1):
    # Tomamos el último dia de cada mes
    last_day = calendar.monthrange(2023, mes)[1]
    starttime = f"2023-{mes}-01"
    endtime = f"2023-{mes}-{last_day}"
    url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={starttime}&endtime={endtime}&minmagnitude=4"
    response = requests.get(url)
    data = json.loads(response.text)
    data = data["features"]
    # Itera sobre cada objeto "Feature" en la lista "data"
    for feature in data:
        # Accede a la parte "properties" de cada objeto
        properties = feature['properties']
        properties['year'] = 2023
        properties['month'] = mes
        # Crea un dataframe de Pandas a partir de la lista de eventos sísmicos
        all_data.append(properties)
            
# Concatenamos todos los dataframes en uno solo
df_global = pd.DataFrame(all_data)

# El código tarda mucho en ejecutarse, habria que optimizarlo. Pero posiblemente la causa es que se esta iterando por año y luego por mes. 
# Por lo que serían muchas iteraciones.

In [31]:
# Observamos
df_global

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,types,nst,dmin,rms,gap,magType,type,title,year,month
0,4.09,"65km ENE of Beatty, NV",2566800720,1454030943020,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",origin,phase-data,",13.0,1.749,0.31,253.0,ml,nuclear explosion,"M 4.1 Nuclear Explosion - 65km ENE of Beatty, NV",1970,1
1,5.90,"59 km NNE of Port-Olry, Vanuatu",2536102930,1652124015110,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,3.894,...,",origin,shakemap,",NaN,NaN,NaN,NaN,mw,earthquake,"M 5.9 - 59 km NNE of Port-Olry, Vanuatu",1970,1
2,6.14,Fiji region,2415962630,1651009857198,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,M 6.1 - Fiji region,1970,1
3,5.27,"195 km WSW of Linxia Chengguanzhen, China",2285370330,1651009855849,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,"M 5.3 - 195 km WSW of Linxia Chengguanzhen, China",1970,1
4,5.77,"38 km SSE of San Cristóbal, Venezuela",2280584330,1651009854455,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,6.276,...,",origin,shakemap,",NaN,NaN,NaN,NaN,mw,earthquake,"M 5.8 - 38 km SSE of San Cristóbal, Venezuela",1970,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461780,5.30,"82 km ESE of Katsuren-haebaru, Japan",1682907747800,1682917939040,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,2.0,2.7,NaN,...,",dyfi,internal-moment-tensor,moment-tensor,ori...",74.0,0.909,0.83,64.0,mww,earthquake,"M 5.3 - 82 km ESE of Katsuren-haebaru, Japan",2023,5
461781,5.10,"Ryukyu Islands, Japan",1682907660774,1682909229040,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",origin,phase-data,",77.0,0.888,1.17,64.0,mb,earthquake,"M 5.1 - Ryukyu Islands, Japan",2023,5
461782,4.30,"45 km NNE of Klyuchi, Russia",1682905534116,1682906478040,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",origin,phase-data,",40.0,3.949,0.63,126.0,mb,earthquake,"M 4.3 - 45 km NNE of Klyuchi, Russia",2023,5
461783,5.10,None,1682900181305,1682902782040,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",origin,phase-data,",66.0,0.682,0.59,121.0,mww,earthquake,M 5.1 -,2023,5


In [32]:
# Observamos más en detalle, se pueden evidenciar columnas con muchos valores faltantes
df_global.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461785 entries, 0 to 461784
Data columns (total 28 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   mag      461785 non-null  float64
 1   place    460393 non-null  object 
 2   time     461785 non-null  int64  
 3   updated  461785 non-null  int64  
 4   tz       2 non-null       float64
 5   url      461785 non-null  object 
 6   detail   461785 non-null  object 
 7   felt     38756 non-null   float64
 8   cdi      38756 non-null   float64
 9   mmi      18486 non-null   float64
 10  alert    7568 non-null    object 
 11  status   461785 non-null  object 
 12  tsunami  461785 non-null  int64  
 13  sig      461785 non-null  int64  
 14  net      461785 non-null  object 
 15  code     461785 non-null  object 
 16  ids      461785 non-null  object 
 17  sources  461785 non-null  object 
 18  types    461785 non-null  object 
 19  nst      176282 non-null  float64
 20  dmin     140673 non-null  

In [39]:
# Vemos mas en detalle
df_global.isna().sum().sort_values(ascending=False).head(10)

tz       461783
alert    454217
mmi      443299
cdi      423029
felt     423029
dmin     321112
nst      285503
gap      188193
rms       55407
place      1392
dtype: int64

In [299]:
# Observamos las columnas, hay algunas que no aportan nada de información útil para el análisis o tienen muchos nulos
df_global.columns

Index(['mag', 'place', 'time', 'updated', 'tz', 'url', 'detail', 'felt', 'cdi',
       'mmi', 'alert', 'status', 'tsunami', 'sig', 'net', 'code', 'ids',
       'sources', 'types', 'nst', 'dmin', 'rms', 'gap', 'magType', 'type',
       'title', 'year', 'month'],
      dtype='object')

In [40]:
# Eliminamos las columnas que no utilizaremos
df_global.drop(['time', 'updated',"url","detail","ids","sources","title","status","tz","alert"],inplace=True,axis=1)

In [42]:
# Observamos que no todos los datos son de terremotos
df_global["type"].value_counts()

earthquake           461014
nuclear explosion       627
volcanic eruption        62
explosion                33
mine collapse            18
quarry blast              9
rock burst                7
mining explosion          7
other event               4
sonic boom                2
landslide                 1
collapse                  1
Name: type, dtype: int64

In [57]:
# Filtramos solo los "earthquake" y reiniciamos el índice
df_global = df_global[df_global["type"] == "earthquake"].reset_index()

In [62]:
# Observamos
df_global

,mag,place,felt,cdi,mmi,tsunami,sig,net,code,types,nst,dmin,rms,gap,magType,type,year,month
0,5.90,"59 km NNE of Port-Olry, Vanuatu",NaN,NaN,3.894,0,536,iscgemsup,800507,",origin,shakemap,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
1,6.14,Fiji region,NaN,NaN,NaN,0,580,iscgem,800471,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
2,5.27,"195 km WSW of Linxia Chengguanzhen, China",NaN,NaN,NaN,0,427,iscgem,800431,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
3,5.77,"38 km SSE of San Cristóbal, Venezuela",NaN,NaN,6.276,0,512,iscgem,800429,",origin,shakemap,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
4,5.72,"20 km ESE of Lata, Solomon Islands",NaN,NaN,NaN,0,503,iscgem,800427,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461009,5.30,"82 km ESE of Katsuren-haebaru, Japan",2.0,2.7,NaN,0,433,us,7000jxaa,",dyfi,internal-moment-tensor,moment-tensor,ori...",74.0,0.909,0.83,64.0,mww,earthquake,2023,5
461010,5.10,"Ryukyu Islands, Japan",NaN,NaN,NaN,0,400,us,7000jxab,",origin,phase-data,",77.0,0.888,1.17,64.0,mb,earthquake,2023,5
461011,4.30,"45 km NNE of Klyuchi, Russia",NaN,NaN,NaN,0,284,us,7000jxa6,",origin,phase-data,",40.0,3.949,0.63,126.0,mb,earthquake,2023,5
461012,5.10,None,NaN,NaN,NaN,0,400,us,7000jx9j,",origin,phase-data,",66.0,0.682,0.59,121.0,mww,earthquake,2023,5


In [64]:
# Exportamos
df_global.to_parquet("Datasets/Usa_Jap_Mex/terremotos_global.parquet",compression="snappy")

---

# Creamos tabla de países

In [65]:
# Creamos un diccionario con los países y los ID de cada uno para la normalización de la base de datos
country = { "idCountry" : [1, 2, 3] ,
             "country" : ["United States", "Japón", "México"]   }
# Creamos un dataframe con estos datos
df_country = pd.DataFrame(country)
# Observamos
df_country

,idCountry,country
0,1,United States
1,2,Japón
2,3,México


In [66]:
# Exportamos
df_country.to_json("Datasets/Usa_Jap_Mex/países_id.json")

---

# USA

In [67]:
# Hacemos una lista de los estados
estados2 = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawai', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
       'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
       'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
       'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin',
       'Wyoming']


In [68]:
# Observamos si hay nulos, ya que estos no nos permitiran filtrar
df_global["place"].isna().sum()

1392

In [69]:
# Rellenamos con "sin dato"
df_global["place"].fillna("sin dato",inplace=True)

In [106]:
# Filtramos en el dataframe global con la lista de estados de Estados Unidos
all_dfs2 = []
for i in estados2 : 
    all_dfs2.append(df_global[df_global["place"].str.contains(i)])

# Luego concatenamos 
df_usa1 = pd.concat(all_dfs2, ignore_index=True)   

In [107]:
# Observamos
df_usa1

,mag,place,felt,cdi,mmi,tsunami,sig,net,code,types,nst,dmin,rms,gap,magType,type,year,month
0,4.50,"3 km NNW of Fayette, Alabama",NaN,NaN,5.447,0,312,us,p0000brg,",origin,shakemap,trump-shakemap,",NaN,NaN,NaN,NaN,mb,earthquake,1975,6
1,4.80,"15 km NNW of Flomaton, Alabama",NaN,NaN,6.012,0,354,us,p00089hq,",impact-text,origin,phase-data,shakemap,trump-...",NaN,NaN,0.98,NaN,mb,earthquake,1997,10
2,4.60,"8 km S of Valley Head, Alabama",17204.0,6.0,4.987,0,926,se,605341,",associate,dyfi,focal-mechanism,impact-text,lo...",21.0,NaN,0.07,154.0,mw,earthquake,2003,4
3,4.30,"4 km W of Forkland, Alabama",85.0,5.0,NaN,0,327,us,p000d7vw,",associate,dyfi,impact-text,moment-tensor,orig...",61.0,NaN,0.90,78.9,mwr,earthquake,2004,11
4,5.73,"Rat Islands, Aleutian Islands, Alaska",NaN,NaN,NaN,0,505,iscgem,800290,",impact-text,origin,trump-origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22350,4.80,"20 km W of Fort Washakie, Wyoming",116.0,4.3,3.570,0,404,us,b000jx4l,",dyfi,losspager,moment-tensor,origin,phase-dat...",NaN,0.37800,1.08,16.0,mwr,earthquake,2013,9
22351,4.83,"22 km S of Mammoth, Wyoming",22.0,2.9,5.570,0,700,uu,60061837,",cap,dyfi,general-link,general-text,losspager,...",35.0,0.04763,0.25,63.0,mw,earthquake,2014,3
22352,4.00,"23 km SE of Kelly, Wyoming",153.0,3.4,4.370,0,298,us,10004t1f,",cap,dyfi,impact-text,losspager,moment-tensor,...",NaN,0.19500,1.16,18.0,mwr,earthquake,2016,2
22353,4.80,"4 km ENE of Bondurant, Wyoming",578.0,3.9,4.640,0,580,us,20006umx,",cap,dyfi,impact-text,losspager,moment-tensor,...",NaN,0.37300,1.14,7.0,mwr,earthquake,2016,8


In [108]:
# Separamos la columna place para tener la distancia y el estado en distintas columnas
df_usa1["place"] = df_usa1["place"].str.strip()
df_usa1[["distance","location","location2"]] = df_usa1["place"].str.split(",",expand=True)
# Borramos la columna location2 y place ya que no nos sirven
df_usa1.drop(["place","location2","type"],inplace=True,axis=1)
# Agregamos el ID correspondiente a USA que es 1
df_usa1["idCountry"] = 1
# Observamos
df_usa1

,mag,felt,cdi,mmi,tsunami,sig,net,code,types,nst,dmin,rms,gap,magType,year,month,distance,location,idCountry
0,4.50,NaN,NaN,5.447,0,312,us,p0000brg,",origin,shakemap,trump-shakemap,",NaN,NaN,NaN,NaN,mb,1975,6,3 km NNW of Fayette,Alabama,1
1,4.80,NaN,NaN,6.012,0,354,us,p00089hq,",impact-text,origin,phase-data,shakemap,trump-...",NaN,NaN,0.98,NaN,mb,1997,10,15 km NNW of Flomaton,Alabama,1
2,4.60,17204.0,6.0,4.987,0,926,se,605341,",associate,dyfi,focal-mechanism,impact-text,lo...",21.0,NaN,0.07,154.0,mw,2003,4,8 km S of Valley Head,Alabama,1
3,4.30,85.0,5.0,NaN,0,327,us,p000d7vw,",associate,dyfi,impact-text,moment-tensor,orig...",61.0,NaN,0.90,78.9,mwr,2004,11,4 km W of Forkland,Alabama,1
4,5.73,NaN,NaN,NaN,0,505,iscgem,800290,",impact-text,origin,trump-origin,",NaN,NaN,NaN,NaN,mw,1970,1,Rat Islands,Aleutian Islands,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22350,4.80,116.0,4.3,3.570,0,404,us,b000jx4l,",dyfi,losspager,moment-tensor,origin,phase-dat...",NaN,0.37800,1.08,16.0,mwr,2013,9,20 km W of Fort Washakie,Wyoming,1
22351,4.83,22.0,2.9,5.570,0,700,uu,60061837,",cap,dyfi,general-link,general-text,losspager,...",35.0,0.04763,0.25,63.0,mw,2014,3,22 km S of Mammoth,Wyoming,1
22352,4.00,153.0,3.4,4.370,0,298,us,10004t1f,",cap,dyfi,impact-text,losspager,moment-tensor,...",NaN,0.19500,1.16,18.0,mwr,2016,2,23 km SE of Kelly,Wyoming,1
22353,4.80,578.0,3.9,4.640,0,580,us,20006umx,",cap,dyfi,impact-text,losspager,moment-tensor,...",NaN,0.37300,1.14,7.0,mwr,2016,8,4 km ENE of Bondurant,Wyoming,1


In [109]:
# Vemos las columnas
df_usa1.columns

Index(['mag', 'felt', 'cdi', 'mmi', 'tsunami', 'sig', 'net', 'code', 'types',
       'nst', 'dmin', 'rms', 'gap', 'magType', 'year', 'month', 'distance',
       'location', 'idCountry'],
      dtype='object')

In [122]:
# Reordenamos para mejor entendimiento
df_usa1 = df_usa1[['idCountry','year','month','location','distance','types', 'code','mag', 'felt', 'cdi', 'mmi', 'tsunami', 'sig', 'net', 'nst', 'dmin', 'rms', 'gap', 'magType']]

# Observamos
df_usa1.sample(5)

,idCountry,year,month,location,distance,types,code,mag,felt,cdi,mmi,tsunami,sig,net,nst,dmin,rms,gap,magType
2874,1,1986,5,Alaska,21 km SW of Atka,",origin,phase-data,",p0002u65,4.0,NaN,NaN,NaN,0,246,us,NaN,NaN,1.10,NaN,mb
18271,1,2013,1,California,252 km SSW of Avalon,",impact-text,nearby-cities,origin,phase-data,",p000jymb,4.0,NaN,NaN,NaN,0,246,us,46.0,NaN,1.01,209.3,md
3936,1,1988,5,Alaska,92 km SW of Atka,",origin,phase-data,",p0003frg,4.7,NaN,NaN,NaN,0,340,us,NaN,NaN,1.40,NaN,mb
13609,1,2017,3,Alaska,25 km NW of Anchor Point,",associate,origin,phase-data,",0173ekfjkt,4.1,NaN,NaN,NaN,0,259,ak,NaN,NaN,0.37,NaN,mb
13401,1,2016,7,Alaska,216 km SE of Akutan,",associate,cap,dyfi,origin,phase-data,",20006fsa,4.4,0.0,1.0,NaN,0,298,us,NaN,1.897,0.83,170.0,mb


In [111]:
# Exportamos
df_usa1.to_json("Datasets/Usa_Jap_Mex/Usa.json")

---

# Japón

In [129]:
# Filtramos el dataframe global con los datos de Japón
df_japon = df_global[df_global["place"].str.contains("Japan")]
# Observamos
df_japon

,mag,place,felt,cdi,mmi,tsunami,sig,net,code,types,nst,dmin,rms,gap,magType,type,year,month
11,6.40,"45 km SSW of Obihiro, Japan",NaN,NaN,6.419,0,630,iscgem,800254,",origin,shakemap,trump-shakemap,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
74,5.53,"Volcano Islands, Japan region",NaN,NaN,NaN,0,470,iscgem,799066,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,2
101,5.54,"67 km NE of Naze, Japan",NaN,NaN,NaN,0,472,iscgem,798599,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,3
119,5.70,"101 km WSW of Kurio, Japan",NaN,NaN,NaN,0,500,iscgem,798320,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,3
121,5.70,"4 km WSW of Takanosu, Japan",NaN,NaN,3.951,0,500,iscgem,798303,",origin,shakemap,",NaN,NaN,NaN,NaN,mw,earthquake,1970,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461005,5.10,"75 km ESE of Katsuren-haebaru, Japan",NaN,NaN,NaN,0,400,us,7000jxal,",origin,phase-data,",58.0,0.883,0.99,79.0,mww,earthquake,2023,5
461006,5.00,"81 km SE of Taira, Japan",NaN,NaN,NaN,0,385,us,7000jxaj,",origin,phase-data,",44.0,0.845,0.95,78.0,mb,earthquake,2023,5
461007,5.90,"79 km ESE of Katsuren-haebaru, Japan",10.0,3.1,3.841,0,539,us,7000jxag,",dyfi,internal-moment-tensor,internal-origin,l...",76.0,0.871,0.75,57.0,mww,earthquake,2023,5
461009,5.30,"82 km ESE of Katsuren-haebaru, Japan",2.0,2.7,NaN,0,433,us,7000jxaa,",dyfi,internal-moment-tensor,moment-tensor,ori...",74.0,0.909,0.83,64.0,mww,earthquake,2023,5


In [136]:
# Agregamos el ID de japon que es 2
df_japon.loc[:, "idCountry"] = 2

In [274]:
# Reordenamos 
df_japon = df_japon[['idCountry','year','month','place','types', 'code','mag', 'felt', 'cdi', 'mmi', 'tsunami', 'sig', 'net', 'nst', 'dmin', 'rms', 'gap', 'magType']]

# Observamos
df_japon.sample(5)

,idCountry,year,month,place,types,code,mag,felt,cdi,mmi,tsunami,sig,net,nst,dmin,rms,gap,magType
458517,2,2023,2,"southwestern Ryukyu Islands, Japan",",origin,phase-data,",6000jn49,4.6,NaN,NaN,NaN,0,326,us,25.0,0.466,0.58,130.0,mb
295247,2,2011,9,"131 km ENE of Miyako, Japan",",impact-text,origin,phase-data,",p000j850,4.4,NaN,NaN,NaN,0,298,us,24.0,NaN,0.70,129.9,mb
370575,2,2016,12,"2 km SSW of Chiba, Japan",",dyfi,impact-text,origin,phase-data,",10007eul,4.7,12.0,2.7,NaN,0,343,us,NaN,1.814,1.02,115.0,mb
203806,2,2004,12,"62 km E of Ishinomaki, Japan",",associate,dyfi,focal-mechanism,impact-text,mo...",p000dc4e,5.7,5.0,3.4,3.785,0,502,us,190.0,NaN,0.93,105.0,mwc
342047,2,2015,2,"Izu Islands, Japan region",",cap,dyfi,losspager,moment-tensor,origin,phase...",c000tt0x,5.9,0.0,1.0,2.950,0,536,us,NaN,2.659,1.28,34.0,mww


In [276]:
# Exportamos
df_japon.to_json("Datasets/Usa_Jap_Mex/Japón.json")

---

# México

In [300]:
# A través del servicio sismológico nacional de México se extraen los siguientes datos a través de una URL
url = "http://www2.ssn.unam.mx:8080/catalogo/csv/20230508125001UBDKO.csv"

# Hacemos una solicitud a la url obteniendo el contenido
response = requests.get(url).content

# StringIO se utiliza para convertir el contenido en una cadena legible por pandas
df_mexico = pd.read_csv(io.StringIO(response.decode('utf-8')), skiprows=4, skipfooter=7, engine='python') # con skiprows salteamos las primeras y últimas filas que generan errores

# Observamos
df_mexico

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Referencia de localizacion,Fecha UTC,Hora UTC,Estatus
0,1950-09-29,00:32:20,7.0,19.00,-107.00,60.0,"257 km al SUROESTE de PUERTO VALLARTA, JAL",1950-09-29,06:32:20,revisado
1,1950-10-21,03:42:58,6.6,17.73,-106.00,33.0,"225 km al SUROESTE de CIHUATLAN, JAL",1950-10-21,09:42:58,revisado
2,1950-10-23,10:13:20,7.2,14.30,-91.80,33.0,"56 km al SURESTE de CD HIDALGO, CHIS",1950-10-23,16:13:20,revisado
3,1950-11-17,13:28:23,6.6,17.50,-100.50,33.0,"33 km al NORTE de ATOYAC DE ALVAREZ, GRO",1950-11-17,19:28:23,revisado
4,1950-12-14,08:15:50,7.2,17.22,-98.12,33.0,"47 km al OESTE de H TLAXIACO, OAX",1950-12-14,14:15:50,revisado
...,...,...,...,...,...,...,...,...,...,...
42724,2023-05-07,19:00:07,4.1,17.47,-104.08,16.7,"160 km al SUR de TECOMAN, COL",2023-05-08,01:00:07,verificado
42725,2023-05-07,21:34:45,4.0,15.41,-94.78,17.1,"96 km al SURESTE de SALINA CRUZ, OAX",2023-05-08,03:34:45,verificado
42726,2023-05-08,01:05:53,4.2,14.87,-94.27,17.4,"145 km al SUROESTE de TONALA, CHIS",2023-05-08,07:05:53,verificado
42727,2023-05-08,01:48:17,4.3,14.54,-92.36,72.7,"27 km al SUROESTE de CD HIDALGO, CHIS",2023-05-08,07:48:17,verificado


In [301]:
# Separamos "referencia de localizacion" en "Distancia" y "Estado"
df_mexico[["Distancia","Estado"]] = df_mexico["Referencia de localizacion"].str.split(",",expand=True)
# Agregamos el ID de México que es 3
df_mexico["idCountry"] = 3
# Observamos
df_mexico.head()

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Referencia de localizacion,Fecha UTC,Hora UTC,Estatus,Distancia,Estado,idCountry
0,1950-09-29,00:32:20,7.0,19.00,-107.00,60.0,"257 km al SUROESTE de PUERTO VALLARTA, JAL",1950-09-29,06:32:20,revisado,257 km al SUROESTE de PUERTO VALLARTA,JAL,3
1,1950-10-21,03:42:58,6.6,17.73,-106.00,33.0,"225 km al SUROESTE de CIHUATLAN, JAL",1950-10-21,09:42:58,revisado,225 km al SUROESTE de CIHUATLAN,JAL,3
2,1950-10-23,10:13:20,7.2,14.30,-91.80,33.0,"56 km al SURESTE de CD HIDALGO, CHIS",1950-10-23,16:13:20,revisado,56 km al SURESTE de CD HIDALGO,CHIS,3
3,1950-11-17,13:28:23,6.6,17.50,-100.50,33.0,"33 km al NORTE de ATOYAC DE ALVAREZ, GRO",1950-11-17,19:28:23,revisado,33 km al NORTE de ATOYAC DE ALVAREZ,GRO,3
4,1950-12-14,08:15:50,7.2,17.22,-98.12,33.0,"47 km al OESTE de H TLAXIACO, OAX",1950-12-14,14:15:50,revisado,47 km al OESTE de H TLAXIACO,OAX,3


In [302]:
# Eliminamos las columnas que no serán de utilidad para el análisis
df_mexico.drop(["Referencia de localizacion","Fecha UTC","Hora UTC","Estatus"],axis=1,inplace=True)
# Observamos
df_mexico

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Distancia,Estado,idCountry
0,1950-09-29,00:32:20,7.0,19.00,-107.00,60.0,257 km al SUROESTE de PUERTO VALLARTA,JAL,3
1,1950-10-21,03:42:58,6.6,17.73,-106.00,33.0,225 km al SUROESTE de CIHUATLAN,JAL,3
2,1950-10-23,10:13:20,7.2,14.30,-91.80,33.0,56 km al SURESTE de CD HIDALGO,CHIS,3
3,1950-11-17,13:28:23,6.6,17.50,-100.50,33.0,33 km al NORTE de ATOYAC DE ALVAREZ,GRO,3
4,1950-12-14,08:15:50,7.2,17.22,-98.12,33.0,47 km al OESTE de H TLAXIACO,OAX,3
...,...,...,...,...,...,...,...,...,...
42724,2023-05-07,19:00:07,4.1,17.47,-104.08,16.7,160 km al SUR de TECOMAN,COL,3
42725,2023-05-07,21:34:45,4.0,15.41,-94.78,17.1,96 km al SURESTE de SALINA CRUZ,OAX,3
42726,2023-05-08,01:05:53,4.2,14.87,-94.27,17.4,145 km al SUROESTE de TONALA,CHIS,3
42727,2023-05-08,01:48:17,4.3,14.54,-92.36,72.7,27 km al SUROESTE de CD HIDALGO,CHIS,3


In [303]:
# Observamos las columnas
df_mexico.columns

Index(['Fecha', 'Hora', 'Magnitud', 'Latitud', 'Longitud', 'Profundidad',
       'Distancia', 'Estado', 'idCountry'],
      dtype='object')

In [305]:
# Reordenamos
df_mexico = df_mexico[['idCountry','Fecha', 'Hora','Distancia', 'Estado', 'Latitud', 'Longitud','Magnitud', 'Profundidad']]
# Observamos
df_mexico

,idCountry,Fecha,Hora,Distancia,Estado,Latitud,Longitud,Magnitud,Profundidad
0,3,1950-09-29,00:32:20,257 km al SUROESTE de PUERTO VALLARTA,JAL,19.00,-107.00,7.0,60.0
1,3,1950-10-21,03:42:58,225 km al SUROESTE de CIHUATLAN,JAL,17.73,-106.00,6.6,33.0
2,3,1950-10-23,10:13:20,56 km al SURESTE de CD HIDALGO,CHIS,14.30,-91.80,7.2,33.0
3,3,1950-11-17,13:28:23,33 km al NORTE de ATOYAC DE ALVAREZ,GRO,17.50,-100.50,6.6,33.0
4,3,1950-12-14,08:15:50,47 km al OESTE de H TLAXIACO,OAX,17.22,-98.12,7.2,33.0
...,...,...,...,...,...,...,...,...,...
42724,3,2023-05-07,19:00:07,160 km al SUR de TECOMAN,COL,17.47,-104.08,4.1,16.7
42725,3,2023-05-07,21:34:45,96 km al SURESTE de SALINA CRUZ,OAX,15.41,-94.78,4.0,17.1
42726,3,2023-05-08,01:05:53,145 km al SUROESTE de TONALA,CHIS,14.87,-94.27,4.2,17.4
42727,3,2023-05-08,01:48:17,27 km al SUROESTE de CD HIDALGO,CHIS,14.54,-92.36,4.3,72.7


In [306]:
# Exportamos a formato json
df_mexico.to_json("Datasets/Usa_Jap_Mex/México.json")

---

# Requerimientos de Data Analysts

---

+ ### KPI 1 

In [ ]:
# Se utilizan los datasets de la carpeta KPI 1

In [307]:
# Leemos el archivo json
df_polizas = pd.read_json("Datasets/KPI 1 y 4/polizas_japon.json")
# Agregamos el ID
df_polizas["idCountry"] = 2
# Observamos
df_polizas

,nombre Estado,Propiedades aseguradas,Monto asegurado,Primas emitidas,Propiedades Aseguradas,Monto asegurado.1,Tipo Construcción,año,idCountry
0,Hokkaido,105197,1370980,2253888,222997,2853620,Tipo A edificios,2019,2
1,Aomori,11104,154461,237145,22744,313867,Tipo A edificios,2019,2
2,Iwate,12400,180140,277996,31708,444239,Tipo A edificios,2019,2
3,Miyagi,57775,854537,1998684,151576,2133825,Tipo A edificios,2019,2
4,Akita,7314,98260,139790,16368,218498,Tipo A edificios,2019,2
...,...,...,...,...,...,...,...,...,...
559,Kumamoto,40154,99993,363675,124323,314686,Tipo B residencias,2021,2
560,Oita,11281,25408,155190,34069,79374,Tipo B residencias,2021,2
561,Miyazaki,14739,32067,198889,44211,103315,Tipo B residencias,2021,2
562,Kagoshima,22104,44871,169119,68987,148932,Tipo B residencias,2021,2


In [309]:
# Leemos el archivo
df_incremento_polizas = pd.read_json("Datasets/KPI 1 y 4/incremento_polizas.json")
# Agregamos el ID
df_incremento_polizas["idCountry"] = 2
# Observamos
df_incremento_polizas

,%,2015,2016,2017,2018,2019,2020,2021,idCountry
0,Hokkaido,51.0,52.4,53.3,56.6,59.1,60.6,61.8,2
1,Aomori,61.8,62.9,63.9,65.5,67.0,68.6,70.1,2
2,Iwate,66.8,67.9,69.1,70.4,72.3,73.7,75.0,2
3,Miyagi,86.2,86.4,86.3,86.8,87.0,87.5,88.7,2
4,Akita,68.5,69.5,70.8,72.0,73.3,74.4,74.8,2
5,Yamagata,60.9,62.5,63.3,64.8,66.3,67.9,68.7,2
6,Fukushima,70.5,72.2,73.1,74.1,75.2,76.7,79.3,2
7,Ibaraki,60.5,61.9,62.2,63.8,64.6,66.0,66.5,2
8,Tochigi,62.2,64.2,65.6,67.6,69.7,71.4,72.7,2
9,Gumma,54.7,56.6,57.6,59.9,62.2,63.9,65.0,2


In [310]:
# Leemos el archivo parquet
seg_mex = pd.read_parquet("Datasets\KPI 1 y 4\seguros_mex.parquet")
# Eliminamos la columna "level_0" que resultó de un error
seg_mex.pop("level_0")
# Agregamos el ID
seg_mex["idCountry"] = 3
# Observamos
seg_mex

,AñO,MONEDA,ZONA SISMICA,TIPO DE SEGURO,SUBTIPO DE SEGURO,NUMERO DE PISOS,NUMERO DE UBICACIONES,PRIMA EMITIDA,PRIMA RETENIDA,PRIMA DEVENGADA,COMISION DIRECTA,VALORES TOTALES EDIFICIO,VALORES TOTALES CONTENIDOS,VALORES TOTALES PeERDIDAS CONSECUENCIALES,LIMITE MAXIMO DE RESPONSABILIDAD,idCountry
0,2019,Nacional,A_,Terremoto,Otro,1,1,0,0,63,0,0,655000,0,655000,3
1,2019,Nacional,B_,Terremoto,Otro,2,2,383,262,382,19,0,1500000,120000,1620000,3
2,2019,Nacional,C_,Terremoto,Otro,1,3,3511,2336,2970,175,5000000,4000000,0,9000000,3
3,2019,Nacional,C_,Terremoto,Otro,2,9,2895,1790,2883,146,2859280,8600000,1150000,12609280,3
4,2019,Nacional,B_,Terremoto,Otro,1,17,6135,331,79334,305,890890000,245448050,81172500,1217510550,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321787,2021,Extranjera,B_,Terremoto,Microseguro,1,1,34337,343,10724,1030,0,306600000,29199991,335799991,3
321788,2021,Extranjera,C_,Terremoto,Microseguro,1,1,4,0,2,0,0,40880,4088,44968,3
321789,2021,Extranjera,B_,Terremoto,Microseguro,1,2,7463,-101,4110,224,0,83804000,8380400,92184400,3
321790,2021,Extranjera,B1,Terremoto,Microseguro,1,4,5809,267,3199,174,0,59547368,7998737,67546105,3


In [311]:
# Exportamos los 3 archivos
df_polizas.to_json("Datasets/KPI 1 y 4/polizas_japon.json")
df_incremento_polizas.to_json("Datasets/KPI 1 y 4/incremento_polizas.json")
seg_mex.to_parquet("Datasets\KPI 1 y 4\seguros_mex.parquet")

---

+ ### KPI 2

In [ ]:
# USA : https://www.worlddata.info/america/usa/earthquakes.php

# Japon : https://www.worlddata.info/asia/japan/earthquakes.php

# Mexico : https://www.worlddata.info/america/mexico/earthquakes.php

In [2]:
# Creamos una variable con los URL para realizar el Web Scrapping
urls = ["https://www.worlddata.info/america/mexico/earthquakes.php", # mexico
        "https://www.worlddata.info/asia/japan/earthquakes.php", # japon
        "https://www.worlddata.info/america/usa/earthquakes.php"] # usa

# Creamos una variable para colocar el ID de cada pais
idCountry = 4

# Creamos una lista donde guardaremos todos los datos
data_total = []

# Iteramos en cada URL 
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    seccion = soup.find('table', {'class': 'std100 hover'})
    rows = seccion.find_all("tr")
    idCountry -= 1
    # Hacemos el web scrapping
    for row in rows[1:]:
        cols = row.find_all('td')
        date = cols[0].text.strip()
        location = cols[1].text.strip()
        depth = cols[2].text.strip()
        magnitude = cols[3].text.strip()
        deaths = cols[4].text.strip()
        idCountry = idCountry
        # Agregamos todo en una lista
        data_total.append([idCountry, date, location, depth, magnitude, deaths])

# Creamos un dataframe con esos datos
df_fatalidades = pd.DataFrame(data_total, columns=["idCountry","date", "location", "depth", "magnitude", "deaths"])  

# Colocamos el tipo de dato correcto en Date
df_fatalidades['date'] = pd.to_datetime(df_fatalidades['date'], format='%m/%d/%Y')

# Observamos
df_fatalidades    

,idCountry,date,location,depth,magnitude,deaths
0,3,2022-09-22,"Mexico City, Michoacan",24 km,6.8,3
1,3,2022-09-19,"Michoacan, Colima, Jalisco",15 km,7.6,2
2,3,2022-05-25,Oaxaca,34 km,5.5,0
3,3,2021-09-08,Guerrero,20 km,7.0,3
4,3,2020-06-23,Oaxaca,26 km,7.4,10
...,...,...,...,...,...,...
350,1,1954-08-23,Stillwater Range,,6.8,0
351,1,1954-07-06,Fallon,,6.8,0
352,1,1952-08-22,Kern County,24 km,5.8,2
353,1,1952-07-21,Kern County,14 km,7.7,12


In [3]:
# Exportamos
df_fatalidades.to_json("Datasets/KPI 2/Fatalidades.json")

---

+ ### KPI 3

In [ ]:
# Web scrapping de estas paginas 

# https://www.shakeout.org/glb_participants.php?year=2022&start=All ( de acá todo)

# https://www.shakeout.org/participants.php?year=2022 ( de acá solo mexico y category)

#No se pudo

In [34]:
from selenium import webdriver
import re

# Cargar la página con Selenium
url = "https://www.shakeout.org/glb_participants.php?year=2022&start=All"
driver = webdriver.Chrome()
driver.get(url)
driver.implicitly_wait(10)

# Ejecutar el código JavaScript en la página
script = """
return [
    createCatNewRow("Individuals/Families", "individuals"),
    createCatNewRow("Childcare and Pre-Schools", "childcare"),
    createCatNewRow("K-12 Schools and Districts", "schools"),
    createCatNewRow("Colleges and Universities", "colleges"),
    createCatNewRow("Local Government", "local"),
    createCatNewRow("State Government", "state"),
    createCatNewRow("Federal/National Government (Including Military)", "federal"),
    createCatNewRow("Tribes/Indigenous Peoples", "tribes"),
    createCatNewRow("Businesses", "businesses"),
    createCatNewRow("Hotels and Other Lodgings", "lodging"),
    createCatNewRow("Healthcare", "healthcare"),
    createCatNewRow("Senior Facilities/Communities", "seniors"),
    createCatNewRow("Disability/AFN Organizations", "disability"),
    createCatNewRow("Non-Profit Organizations", "nonprofits"),
    createCatNewRow("Neighborhood Groups", "neighborhoods"),
    createCatNewRow("Preparedness Organizations", "preparedness"),
    createCatNewRow("Faith-based Organizations", "faithbased"),
    createCatNewRow("Museums, Libraries, Parks, etc.", "museums"),
    createCatNewRow("Volunteer/Service Clubs", "service"),
    createCatNewRow("Youth Organizations", "youth"),
    createCatNewRow("Animal Shelter/Service Providers", "animalservices"),
    createCatNewRow("Agriculture/Livestock Sector", "agriculture"),
    createCatNewRow("Volunteer Radio Groups", "radiogroups"),
    createCatNewRow("Science/Engineering Organizations", "science"),
    createCatNewRow("Media Organizations", "media"),
    createCatNewRow("Other", "other")
];
"""
categories = driver.execute_script(script)

# Cerrar el navegador
driver.quit()

# Imprimir las categorías
print(categories)


['<tr><td align="left">Individuals/Families</td><td align="right" style="font-size:14px;">21,522</td></tr>', '<tr><td align="left">Childcare and Pre-Schools</td><td align="right" style="font-size:14px;">152,549</td></tr>', '<tr><td align="left">K-12 Schools and Districts</td><td align="right" style="font-size:14px;">14,187,763</td></tr>', '<tr><td align="left">Colleges and Universities</td><td align="right" style="font-size:14px;">2,827,494</td></tr>', '<tr><td align="left">Local Government</td><td align="right" style="font-size:14px;">1,396,314</td></tr>', '<tr><td align="left">State Government</td><td align="right" style="font-size:14px;">6,273,225</td></tr>', '<tr><td align="left">Federal/National Government (Including Military)</td><td align="right" style="font-size:14px;">11,340,754</td></tr>', '<tr><td align="left">Tribes/Indigenous Peoples</td><td align="right" style="font-size:14px;">11,894</td></tr>', '<tr><td align="left">Businesses</td><td align="right" style="font-size:14px

In [37]:
rows = categories
# expresión regular para extraer la categoría y el valor numérico de una fila
pattern = re.compile(r'<tr><td align="left">(.*?)</td><td align="right" style="font-size:14px;">(.*?)</td></tr>')

# iterar sobre cada fila y extraer la categoría y el valor numérico
lista = []
for row in rows:
    match = pattern.search(row)
    if match:
        category = match.group(1)
        value = match.group(2)
        lista.append((category, value))

print(lista)


[('Individuals/Families', '21,522'), ('Childcare and Pre-Schools', '152,549'), ('K-12 Schools and Districts', '14,187,763'), ('Colleges and Universities', '2,827,494'), ('Local Government', '1,396,314'), ('State Government', '6,273,225'), ('Federal/National Government (Including Military)', '11,340,754'), ('Tribes/Indigenous Peoples', '11,894'), ('Businesses', '562,973'), ('Hotels and Other Lodgings', '6,785'), ('Healthcare', '646,717'), ('Senior Facilities/Communities', '75,587'), ('Disability/AFN Organizations', '14,137'), ('Non-Profit Organizations', '1,880,922'), ('Neighborhood Groups', '25,557'), ('Preparedness Organizations', '42,686'), ('Faith-based Organizations', '188,996'), ('Museums, Libraries, Parks, etc.', '9,436'), ('Volunteer/Service Clubs', '15,415'), ('Youth Organizations', '5,057'), ('Animal Shelter/Service Providers', '433'), ('Agriculture/Livestock Sector', '2,667'), ('Volunteer Radio Groups', '4,183'), ('Science/Engineering Organizations', '11,893'), ('Media Organi

---

+ ### KPI 4 

In [ ]:
# Utiliza los mismos Datasets que el KPI 1.

---

+ ### KPI 5

In [ ]:
# https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?locations=US-JP-MX

In [147]:
# Ingestaremos los datos a través de una URL
url = "https://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.CD?downloadformat=excel"
df_gdp = pd.read_excel(url)
# Observamos, se puede evidenciar que los nombres de las columnas estan mal posicionados
df_gdp.head()

,Data Source,World Development Indicators,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65
0,Last Updated Date,2023-03-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Country Name,Country Code,Indicator Name,Indicator Code,1.960000e+03,1.961000e+03,1.962000e+03,1.963000e+03,1.964000e+03,1.965000e+03,...,2.012000e+03,2.013000e+03,2.014000e+03,2.015000e+03,2.016000e+03,2.017000e+03,2.018000e+03,2.019000e+03,2.020000e+03,2.021000e+03
3,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.615084e+09,2.727933e+09,2.791061e+09,2.963128e+09,2.983799e+09,3.092179e+09,3.202235e+09,3.368970e+09,2.610039e+09,3.126019e+09
4,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2.129152e+10,2.180944e+10,2.370806e+10,2.821128e+10,2.611994e+10,2.968348e+10,...,9.720022e+11,9.826771e+11,1.003403e+12,9.231439e+11,8.898593e+11,1.030482e+12,1.016697e+12,1.009052e+12,9.341791e+11,1.089454e+12


In [148]:
# Corregimos las columnas
df_gdp.columns = df_gdp.iloc[2]
# Eliminamos las filas
df_gdp = df_gdp.drop([0,1,2]) 
# Reiniciamos el índice
df_gdp = df_gdp.reset_index()
# Como podemos observar nos da datos de todos los países y solo precisamos Usa, Japón y México para este análisis
df_gdp.head(3)

2,index,Country Name,Country Code,Indicator Name,Indicator Code,1960.0,1961.0,1962.0,1963.0,1964.0,...,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0
0,3,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,...,2.615084e+09,2.727933e+09,2.791061e+09,2.963128e+09,2.983799e+09,3.092179e+09,3.202235e+09,3.368970e+09,2.610039e+09,3.126019e+09
1,4,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2.129152e+10,2.180944e+10,2.370806e+10,2.821128e+10,2.611994e+10,...,9.720022e+11,9.826771e+11,1.003403e+12,9.231439e+11,8.898593e+11,1.030482e+12,1.016697e+12,1.009052e+12,9.341791e+11,1.089454e+12
2,5,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,...,2.020357e+10,2.056449e+10,2.055058e+10,1.999816e+10,1.801956e+10,1.889635e+10,1.841885e+10,1.890449e+10,2.014344e+10,1.478686e+10


In [149]:
# Filtramos los países de interés para nuestro análisis
df_gdp = df_gdp.loc[df_gdp["Country Name"].isin(["United States", "Japan", "Mexico"])].reset_index()
# Observamos
df_gdp

2,level_0,index,Country Name,Country Code,Indicator Name,Indicator Code,1960.0,1961.0,1962.0,1963.0,...,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0
0,119,122,Japan,JPN,GDP (current US$),NY.GDP.MKTP.CD,4.430734e+10,5.350862e+10,6.072302e+10,6.949813e+10,...,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12,5.003678e+12,4.930837e+12,5.037835e+12,5.123318e+12,5.040108e+12,4.940878e+12
1,154,157,Mexico,MEX,GDP (current US$),NY.GDP.MKTP.CD,1.304000e+10,1.416000e+10,1.520000e+10,1.696000e+10,...,1.201090e+12,1.274443e+12,1.315351e+12,1.171868e+12,1.078491e+12,1.158913e+12,1.222408e+12,1.269012e+12,1.090515e+12,1.272839e+12
2,251,254,United States,USA,GDP (current US$),NY.GDP.MKTP.CD,5.433000e+11,5.633000e+11,6.051000e+11,6.386000e+11,...,1.625397e+13,1.684319e+13,1.755068e+13,1.820602e+13,1.869511e+13,1.947734e+13,2.053306e+13,2.138098e+13,2.106047e+13,2.331508e+13


In [150]:
# Eliminamos las columnas que no son de utilidad
df_gdp.drop(["level_0","index","Indicator Code","Indicator Name"],axis=1,inplace=True)
# Observamos
df_gdp

2,Country Name,Country Code,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,1966.0,1967.0,...,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0
0,Japan,JPN,4.430734e+10,5.350862e+10,6.072302e+10,6.949813e+10,8.174901e+10,9.095028e+10,1.056281e+11,1.237819e+11,...,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12,5.003678e+12,4.930837e+12,5.037835e+12,5.123318e+12,5.040108e+12,4.940878e+12
1,Mexico,MEX,1.304000e+10,1.416000e+10,1.520000e+10,1.696000e+10,2.008000e+10,2.184000e+10,2.432000e+10,2.656000e+10,...,1.201090e+12,1.274443e+12,1.315351e+12,1.171868e+12,1.078491e+12,1.158913e+12,1.222408e+12,1.269012e+12,1.090515e+12,1.272839e+12
2,United States,USA,5.433000e+11,5.633000e+11,6.051000e+11,6.386000e+11,6.858000e+11,7.437000e+11,8.150000e+11,8.617000e+11,...,1.625397e+13,1.684319e+13,1.755068e+13,1.820602e+13,1.869511e+13,1.947734e+13,2.053306e+13,2.138098e+13,2.106047e+13,2.331508e+13


In [151]:
# Hay muchas columnas, podemos aplicar un pandas.melt para solucionarlo
df_gdp = df_gdp.melt(["Country Name","Country Code"],value_name="gdp(current US$)")
# Observamos 
df_gdp

,Country Name,Country Code,2,gdp(current US$)
0,Japan,JPN,1960.0,4.430734e+10
1,Mexico,MEX,1960.0,1.304000e+10
2,United States,USA,1960.0,5.433000e+11
3,Japan,JPN,1961.0,5.350862e+10
4,Mexico,MEX,1961.0,1.416000e+10
...,...,...,...,...
181,Mexico,MEX,2020.0,1.090515e+12
182,United States,USA,2020.0,2.106047e+13
183,Japan,JPN,2021.0,4.940878e+12
184,Mexico,MEX,2021.0,1.272839e+12


In [152]:
# Normalizamos los nombres de las columnas
df_gdp.rename(columns={"Country Name" : "country", "Country Code" : "code", "Indicator Name" : "indicator", 2 : "year"},inplace=True)
# Agregamos el ID para cada país
df_gdp["idCountry"] = df_gdp["country"].replace({'United States': 1, 'Japan': 2, 'Mexico': 3})
# Observamos
df_gdp

,country,code,year,gdp(current US$),idCountry
0,Japan,JPN,1960.0,4.430734e+10,2
1,Mexico,MEX,1960.0,1.304000e+10,3
2,United States,USA,1960.0,5.433000e+11,1
3,Japan,JPN,1961.0,5.350862e+10,2
4,Mexico,MEX,1961.0,1.416000e+10,3
...,...,...,...,...,...
181,Mexico,MEX,2020.0,1.090515e+12,3
182,United States,USA,2020.0,2.106047e+13,1
183,Japan,JPN,2021.0,4.940878e+12,2
184,Mexico,MEX,2021.0,1.272839e+12,3


In [153]:
# Borramos la columna country, para garantizar una forma normal
df_gdp.pop("country")
# Reordenamos
df_gdp = df_gdp[["idCountry","code","year","gdp(current US$)"]]
# Observamos
df_gdp

,idCountry,code,year,gdp(current US$)
0,2,JPN,1960.0,4.430734e+10
1,3,MEX,1960.0,1.304000e+10
2,1,USA,1960.0,5.433000e+11
3,2,JPN,1961.0,5.350862e+10
4,3,MEX,1961.0,1.416000e+10
...,...,...,...,...
181,3,MEX,2020.0,1.090515e+12
182,1,USA,2020.0,2.106047e+13
183,2,JPN,2021.0,4.940878e+12
184,3,MEX,2021.0,1.272839e+12


In [154]:
df_gdp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   idCountry         186 non-null    int64  
 1   code              186 non-null    object 
 2   year              186 non-null    object 
 3   gdp(current US$)  186 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 5.9+ KB


In [155]:
# Colocamos el tipo de dato correcto
df_gdp['idCountry'] = df_gdp['idCountry'].astype('int64')
df_gdp['year'] = df_gdp['year'].astype('int64')
df_gdp['gdp(current US$)'] = df_gdp['gdp(current US$)'].astype('float64')
# Observamos
df_gdp

,idCountry,code,year,gdp(current US$)
0,2,JPN,1960,4.430734e+10
1,3,MEX,1960,1.304000e+10
2,1,USA,1960,5.433000e+11
3,2,JPN,1961,5.350862e+10
4,3,MEX,1961,1.416000e+10
...,...,...,...,...
181,3,MEX,2020,1.090515e+12
182,1,USA,2020,2.106047e+13
183,2,JPN,2021,4.940878e+12
184,3,MEX,2021,1.272839e+12


In [447]:
# Exportamos a formato JSON para su posterior análisis
df_gdp.to_json("Datasets/KPI 5/GDP (1960 - 2021).json")

---